In [1]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os, sys, time
import tensorflow as tf
from tensorflow import keras

## **2.1 数据集构建**

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_all, y_train_all), (X_test, y_test) = fashion_mnist.load_data()
X_valid, X_train = X_train_all[:5000], X_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
std_scaler.fit(X_train.astype(np.float32).reshape(-1, 1))
X_train_scaled = std_scaler.transform(X_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
X_valid_scaled = std_scaler.transform(X_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
y_train = np.asarray(y_train, np.int64)

In [3]:
def make_dataset(images, labels, epochs, batch_size, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [4]:
batch_size = 20
epochs = 10
images_placeholder = tf.placeholder(tf.float64, [None, 28, 28])
labels_placeholder = tf.placeholder(tf.int64, [None,])
dataset = make_dataset(images_placeholder, labels_placeholder, 
                       epochs = epochs,
                       batch_size = batch_size)
dataset_iter = dataset.make_initializable_iterator()
x, y = dataset_iter.get_next()
y = tf.cast(y, tf.int64)

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


## **2.2 模型搭建**

In [5]:
hidden_units = [100, 100]
class_num = 10


input_for_next_layer = tf.reshape(x, (-1, 784))
for hidden_unit in hidden_units:
    input_for_next_layer = tf.layers.dense(input_for_next_layer, hidden_unit, activation=tf.nn.relu)
logits = tf.layers.dense(input_for_next_layer, class_num)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=logits)
predictions = tf.argmax(logits, axis=1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(y, predictions), tf.float64))

train_op = tf.train.AdadeltaOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
print(x)
print(logits)

Tensor("IteratorGetNext:0", shape=(?, 28, 28), dtype=float64)
Tensor("dense_2/BiasAdd:0", shape=(?, 10), dtype=float64)


## **2.3 训练**

In [10]:
init = tf.global_variables_initializer()
train_steps_per_epoch = X_train.shape[0] // batch_size


with tf.Session() as sess:
    sess.run([init, dataset_iter.initializer], feed_dict={images_placeholder: X_train_scaled, 
                                                                                       labels_placeholder: y_train})
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
            _, acc_train, loss_train = sess.run([train_op, accuracy, loss])
        print(f'[TRAIN] acc:{acc_train:.4f}, loss:{loss_train:.4f}')

[TRAIN] acc:0.0500, loss:2.4292
[TRAIN] acc:0.3000, loss:2.0575
[TRAIN] acc:0.4000, loss:1.6803
[TRAIN] acc:0.7500, loss:1.3852
[TRAIN] acc:0.6500, loss:1.2697
[TRAIN] acc:0.5500, loss:1.3020
[TRAIN] acc:0.9000, loss:0.8882
[TRAIN] acc:0.6500, loss:1.1555
[TRAIN] acc:0.5000, loss:1.3353
[TRAIN] acc:0.9000, loss:0.9642


辨析**make_one_shot_iterator()** 和 **make_initializable_iterator()**      
前者是不能重复初始化数据的，而后者可以重复初始化数据，所以以placeholder作为输入的话，是需要用后者的   